In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Gen AI Evaluation SDK for Google Observability Gen AI multi-modal datasets

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fevaluating_observability_datasets.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/evaluating_observability_datasets.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_observability_datasets.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Author |
| --- |
| [Matthew Yun](https://github.com/matty3) |

## Overview

This notebook demonstrates how users can use the [Vertex AI SDK Gen AI evaluation service](https://github.com/googleapis/python-aiplatform) for evaluating your Gen AI multimodal content stored in [Google Observability](https://cloud.google.com/stackdriver/docs).

The Vertex AI SDK allows users to run evaluations on their Gen AI models prompts and responses. This now includes the ability for users to run evaluations against the Gen AI data that has been stored in Google Cloud Storage (GCS) within Observability following [OpenTelemetry semantic conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/).

Prompt, response, and system instruction data in Google Observability are stored in separate GCS references. This notebook gives an example for how users can read in their data from GCS and run an evaluation using the Vertex AI SDK.

## Get started

### Install Vertex AI SDK for Gen AI Evaluation Service


In [ ]:
%pip install --upgrade "google-cloud-aiplatform[evaluation]>=1.122.0" --force-reinstall --quiet --no-warn-conflicts

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

# fmt: off
PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
LOCATION = "us-central1"  # @param {type: "string", placeholder: "us-central1", isTemplate: true}
# fmt: on
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", LOCATION)

from vertexai import Client, types

client = Client(project=PROJECT_ID, location=LOCATION)

## Evaluation Dataset

### Load in Google Observability Gen AI dataset

We will need to read in the data stored in Google Cloud Storage and prepare for evaluation.

In [ ]:
# fmt: off
INPUT_SOURCE = ""  # @param {type: "string", placeholder: "[your-input-source]", isTemplate: true}
OUTPUT_SOURCE = ""  # @param {type: "string", placeholder: "[your-output-source]", isTemplate: true}
SYSTEM_INSTRUCTION_SOURCE = ""  # @param {type: "string", placeholder: "[your-system-instruction-source]", isTemplate: true}
# fmt: on

eval_case = types.ObservabilityEvalCase(
    input_src=INPUT_SOURCE,
    output_src=OUTPUT_SOURCE,
    system_instruction_src=SYSTEM_INSTRUCTION_SOURCE,
    api_client=client,
)
eval_dataset = types.EvaluationDataset.load_from_observability_eval_cases([eval_case])

eval_dataset.show()

### Run Evaluation

Evaluate the responses using the `GENERAL_QUALITY` adaptive rubric-based metric by default.

In [ ]:
eval_result = client.evals.evaluate(dataset=eval_dataset)

eval_result.show()